<a href="https://colab.research.google.com/github/mickyp/binance-profit-calculator/blob/main/%5BEmpty%5D_%E5%B9%A3%E5%AE%89%E6%90%8D%E7%9B%8A%E8%A8%88%E7%AE%97%E6%A9%9F_Binance_Profit_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 作者 Micky Lee (mickyp100 @ gmail)
# 日期 2021.12.10
# 專案 https://github.com/mickyp/binance-profit-calculator
# 參考
# 1. https://algotrading101.com/learn/binance-python-api-guide/
# 2. https://python-binance.readthedocs.io/en/latest/


In [ ]:
pip install python-binance

In [ ]:
import os
#import btalib
import pandas as pd
import datetime

from binance.client import Client

In [ ]:
# init
api_key = '填入幣安的 api key'
api_secret = '填入幣安的 api secret'

In [ ]:
client = Client(api_key, api_secret)
frames = []

#台幣對美金匯率
twd = 28

In [ ]:
def bitcoinInfo(symbol1, symbol2):
  pair = symbol1 + symbol2
  print('*****  ', symbol1, '/', symbol2, '  *****')
  trades = client.get_my_trades(symbol=pair)

  # 取得目前 ticker 價錢
  ticker_price = client.get_symbol_ticker(symbol=pair)
  ticker_price = float(ticker_price['price'])
  print('當前價位：', ticker_price)

  # 設置計算結果
  total_df = pd.DataFrame(columns=['symbol1','symbol2','當前價位','持有成本','數量','總價','損益%','美金損益','台幣損益'])

  if len(trades) == 0: # 沒有交易紀錄，代表不是經由購買的管道，而是透過礦池或質押的方式取得
    
    balance = client.get_asset_balance(asset=symbol1)
    currentBalance = float(balance['free']) + float(balance['locked'])
    print('非交易而來，故無購入成本')
    currentTotal = currentBalance * ticker_price
    print('目前持有成本：', ticker_price, ', 持有數量：', currentBalance, ', 持有總價：', currentTotal, symbol2)
    new_row = [symbol1, symbol2, ticker_price, ticker_price, currentBalance, currentTotal, 0, currentTotal, currentTotal*28]
    total_df.loc[len(total_df)] = new_row
  else:
    # 代表是經由購買的管道，所以有交易紀錄
    trades_df = pd.DataFrame(trades)

    # clean data
    df = trades_df[['symbol', 'price', 'qty', 'quoteQty', 'isBuyer', 'isMaker']]
    df = df.astype({'price':'float', 'qty':'float', 'quoteQty':'float',})
    df.head()

    # 計算買入的平均成本
    df_buy = df[df['isBuyer'] == True]
    if df_buy.size > 0:
      df_buy_sum = df_buy.sum()
      df_buy_sum_quote_qty = df_buy_sum['quoteQty']
      df_buy_sum_qty = df_buy_sum['qty']
      #df_buy_cost = df_buy_sum['quoteQty']/df_buy_sum['qty']
      df_buy_cost = df_buy_sum_quote_qty/df_buy_sum_qty
      print('購入平均成本：', df_buy_cost, ', 持有數量：', df_buy_sum_qty, ', 購入總價: ', df_buy_sum_quote_qty, symbol2)
    else:
      df_buy_sum_quote_qty = 0
      df_buy_sum_qty = 0

    # 計算賣出的平均成本
    df_sell = df[df['isBuyer'] == False]
    if df_sell.size > 0:
      df_sell_sum = df_sell.sum()
      df_sell_sum_quote_qty = df_sell_sum['quoteQty']
      df_sell_sum_qty = df_sell_sum['qty']
      #df_sell_cost = df_sell_sum['quoteQty']/df_sell_sum['qty']
      df_sell_cost = df_sell_sum_quote_qty/df_sell_sum_qty
      print('賣出平均成本：', df_sell_cost, ', 賣出數量：', df_sell_sum_qty, ', 賣出總價: ', df_sell_sum_quote_qty, symbol2)
    else:
      df_sell_sum_quote_qty = 0
      df_sell_sum_qty = 0

    # 計算剩下的購入成本
    #if df_sell.size > 0:
    now_qty = df_buy_sum_qty - df_sell_sum_qty # 剩餘數量
    now_quoteQty = df_buy_sum_quote_qty - df_sell_sum_quote_qty # 剩餘付出
    if now_qty < 0:
      now_qty = 0
    if now_quoteQty < 0:
      now_quoteQty = 0
      print(pair, '已回本')
    
    # 如果現在有持有，才去計算持有成本
    if now_qty > 0:
      now_cost = now_quoteQty/now_qty
      if now_quoteQty <= 0:
        now_cost = df_buy_cost  #因為已回本了，所以用當初購入的初始成本
      print('目前持有成本：', now_cost, ', 持有數量：', now_qty, ', 持有總價：', now_quoteQty, symbol2)
    else:
      now_cost = 0
    
    print('---')
    
    # 計算獲利率
    if now_cost > 0:
      profit = (ticker_price - now_cost)/now_cost*100
      print('獲利:', profit, '%')
    else:
      profit = 0

    # 計算損益
    deltaUSDT = ticker_price*now_qty - now_cost*now_qty
    
    # 如果 symbol2 不是 USDT，則將其換算成 USDT
    if symbol2 != 'USDT':
      transfer_price = client.get_symbol_ticker(symbol=(symbol2+'USDT'))
      transfer_price = float(transfer_price['price'])
      print('將', symbol2 , '轉成 USDT, 價位是', transfer_price)
      deltaUSDT = deltaUSDT * transfer_price

    deltaTWD = deltaUSDT * twd
    print('損益:', deltaUSDT, 'USDT')
    print('損益:', deltaTWD, 'TWD')

    new_row = [symbol1, symbol2, ticker_price, now_cost, now_qty, now_quoteQty, profit, deltaUSDT, deltaTWD]
    total_df.loc[len(total_df)] = new_row

  print()
  frames.append(total_df)


In [ ]:
# 輸入你曾經有買賣過的交易對，可以輸入多組，每一組之間要換行

# BTC/USDT 交易對
bitcoinInfo('BTC', 'USDT')

# ETH/USDT 交易對
bitcoinInfo('ETH', 'USDT')

*****   BTC / USDT   *****
當前價位： 48256.9
購入平均成本： 53898.087870840856 , 持有數量： 0.061009999999999995 , 購入總價:  3288.3223410000005 USDT
賣出平均成本： 61229.99999999999 , 賣出數量： 0.005 , 賣出總價:  306.15 USDT
目前持有成本： 53243.56973754687 , 持有數量： 0.05601 , 持有總價： 2982.1723410000004 USDT
---
獲利: -9.365768978540743 %
損益: -279.3033720000003 USDT
損益: -7820.494416000009 TWD

*****   ETH / USDT   *****
當前價位： 4119.89
購入平均成本： 3843.2987378210805 , 持有數量： 0.2258 , 購入總價:  867.816855 USDT
目前持有成本： 3843.2987378210805 , 持有數量： 0.2258 , 持有總價： 867.816855 USDT
---
獲利: 7.196715141007499 %
損益: 62.454307000000085 USDT
損益: 1748.7205960000024 TWD



In [ ]:
all = pd.concat(frames)
print('***** 所有 [交易對] 的損益一覽表 *****')
all

***** 所有 [交易對] 的損益一覽表 *****


,symbol1,symbol2,當前價位,持有成本,數量,總價,損益%,美金損益,台幣損益
0,BTC,USDT,48256.90,53243.569738,0.05601,2982.172341,-9.365769,-279.303372,-7820.494416
0,ETH,USDT,4119.89,3843.298738,0.22580,867.816855,7.196715,62.454307,1748.720596


In [ ]:
print('***** 資金摘要 *****')

# 取得入金記錄
deposit = client.get_deposit_history()
deposit_df = pd.DataFrame(deposit)

# clean data
deposit_df = deposit_df[['amount', 'coin', 'insertTime']]
deposit_df = deposit_df.astype({'amount':'float'})

deposit_sum = deposit_df.sum()
print('幣安已入金：', deposit_sum['amount'], 'USDT')

all_sum = all.sum()
print('美金損益：', all_sum['美金損益'], 'USDT')
print('台幣損益：', all_sum['台幣損益'], 'TWD')

***** 資金摘要 *****
幣安已入金： 7784.5 USDT
美金損益： -216.84906500000022 USDT
台幣損益： -6071.773820000006 TWD
